<a href="https://colab.research.google.com/github/y-arjun-y/liars-miller-rabin/blob/main/liars_miller_rabin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Liars in the Miller-Rabin Primality Test by Arjun Yadav
## All code is available on [GitHub](https://github.com/y-arjun-y/liars-miller-rabin).

## Implementation of the Miller-Rabin primality test in Python

In [1]:
'''
Function by https://rosettacode.org/wiki/Miller%E2%80%93Rabin_primality_test#Python with slight changes to support a custom witness and explanation of the code. 
Function and its formula is based on https://en.wikipedia.org/wiki/Miller%E2%80%93Rabin_primality_test.
A return value of False means n in not prime (i.e. composite).
A return value of True means n is very likely to be a prime (depending on the number of rounds).
'''

def miller_rabin_base(n, a):
  # checking if it is an integer
    if n != int(n):
        return False

    n = int(n)

    # base cases
    if n == 0 or n == 1 or n == 4 or n == 6 or n == 8 or n == 9:
        return False

    if n == 2 or n == 3 or n == 5 or n == 7:
        return True

    if n % 2 == 0:
        return False

    # assigning variables
    s = 0
    d = n - 1

    while d % 2 == 0:
        d >>= 1
        s += 1
    
    assert(2**s * d == n - 1)

    # trial run
    def trial_composite(a):
        if pow(a, d, n) == 1:
            return False

        for i in range(s):
            if pow(a, 2**i * d, n) == n - 1:
                return False
        return True

    # number of trials, directly proportional to time and accuracy.
    num_trials = 10

    # true run
    for _ in range(num_trials):
        if trial_composite(a):
            return False

    return True

## Supporting multiple witnesses in the Miller-Rabin primality test

In [2]:
'''
Returns False if any one witness returns False and True if all witnesses return True.
Based on the following facts about the Miller-Rabin primality test:
  A return value of False means n in not prime (i.e. composite).
  A return value of True means n is very likely to be a prime (depending on the number of rounds).
'''

def miller_rabin(num, witness_list):
  result = []
  
  for i in range(len(witness_list)):
    result.append(miller_rabin_base(num, witness_list[i]))
  
  if False in result:
    return False
  
  return True

## Simple prime number test

In [3]:
'''
Returns whether a number is prime (true) or not (false).
Credit: https://en.wikipedia.org/wiki/Primality_test#Python
'''

def actual_prime(n):
    if n <= 3:
        return n > 1
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i ** 2 <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

## Finding the worst numbers in the Miller-Rabin primality test (for `n` in a certain range, single witness)

In [61]:
# imports
from tqdm import tqdm

# constants
DEFENDANT_RANGE = 1000
DEFENDANTS = [i for i in range(DEFENDANT_RANGE + 1) if i == 2 or i % 2 != 0]
if DEFENDANT_RANGE % 2 == 0:
  WITNESS_RANGE = DEFENDANT_RANGE - 1
else:
  WITNESS_RANGE = DEFENDANT_RANGE

# variable
results = {}

# trial loop function
def trial_loop(defendants, witness):
  lie_count = 0

  for defendant in defendants:
      if miller_rabin_base(defendant, witness) != actual_prime(defendant):
        lie_count += 1
  
  return lie_count

# main loop
for i in tqdm(range(WITNESS_RANGE)):
  results[i] = trial_loop(DEFENDANTS, i)

# finding top 25 worst witnesses
for i in sorted(results.items(), key=lambda x: x[1], reverse=True)[:25]:
  print(i)

100%|██████████| 999/999 [00:08<00:00, 117.30it/s]

(1, 331)
(0, 164)
(676, 18)
(901, 18)
(374, 17)
(944, 16)
(766, 15)
(824, 15)
(900, 15)
(946, 15)
(989, 15)
(293, 14)
(307, 14)
(524, 14)
(557, 14)
(584, 14)
(638, 14)
(734, 14)
(526, 13)
(562, 13)
(566, 13)
(568, 13)
(776, 13)
(818, 13)
(874, 13)


## Finding the worst numbers in the Miller-Rabin primality test (for `n` in a certain range, 2 witnesses)

In [87]:
# imports
from tqdm import tqdm
import itertools

# constant
DEFENDANT_RANGE = 100
DEFENDANTS = [i for i in range(DEFENDANT_RANGE + 1) if i == 2 or i % 2 != 0]
if DEFENDANT_RANGE % 2 == 0:
  WITNESS_RANGE = DEFENDANT_RANGE - 1
else:
  WITNESS_RANGE = DEFENDANT_RANGE
WITNESSES = list(itertools.product([i for i in range(WITNESS_RANGE)], [i for i in range(WITNESS_RANGE)]))

# variables
results = {}

# trial loop function
def trial_loop(defendants, witnesses):
  lie_count = 0

  for defendant in defendants:
      if miller_rabin(defendant, [witnesses[0], witnesses[1]]) != actual_prime(defendant):
        lie_count += 1
  
  return lie_count

# main loop
for i in tqdm(WITNESSES):
  results[i] = trial_loop(DEFENDANTS, (i))

# finding top 25 worst witnesses
for i in sorted(results.items(), key=lambda x: x[1], reverse=True)[:25]:
  print(i)

100%|██████████| 9801/9801 [00:15<00:00, 621.27it/s]

((1, 1), 24)
((0, 0), 21)
((0, 1), 21)
((0, 2), 21)
((0, 3), 21)
((0, 4), 21)
((0, 5), 21)
((0, 6), 21)
((0, 7), 21)
((0, 8), 21)
((0, 9), 21)
((0, 10), 21)
((0, 11), 21)
((0, 12), 21)
((0, 13), 21)
((0, 14), 21)
((0, 15), 21)
((0, 16), 21)
((0, 17), 21)
((0, 18), 21)
((0, 19), 21)
((0, 20), 21)
((0, 21), 21)
((0, 22), 21)
((0, 23), 21)


## Finding the worst numbers in the Miller-Rabin primality test (for `n` in a certain range, 3 witnesses)

In [85]:
# imports
from tqdm import tqdm
import itertools

# constant
DEFENDANT_RANGE = 50
DEFENDANTS = [i for i in range(DEFENDANT_RANGE + 1) if i == 2 or i % 2 != 0]
if DEFENDANT_RANGE % 2 == 0:
  WITNESS_RANGE = DEFENDANT_RANGE - 1
else:
  WITNESS_RANGE = DEFENDANT_RANGE
WITNESSES = list(itertools.product([i for i in range(WITNESS_RANGE)], [i for i in range(WITNESS_RANGE)], [i for i in range(WITNESS_RANGE)]))

# variables
results = {}

# trial loop function
def trial_loop(defendants, witnesses):
  lie_count = 0

  for defendant in defendants:
      if miller_rabin(defendant, [witnesses[0], witnesses[1]]) != actual_prime(defendant):
        lie_count += 1
  
  return lie_count

# main loop
for i in tqdm(WITNESSES):
  results[i] = trial_loop(DEFENDANTS, (i))

# finding top 25 worst witnesses
for i in sorted(results.items(), key=lambda x: x[1], reverse=True)[:25]:
  print(i)

100%|██████████| 117649/117649 [01:29<00:00, 1320.72it/s]


((0, 0, 0), 11)
((0, 0, 1), 11)
((0, 0, 2), 11)
((0, 0, 3), 11)
((0, 0, 4), 11)
((0, 0, 5), 11)
((0, 0, 6), 11)
((0, 0, 7), 11)
((0, 0, 8), 11)
((0, 0, 9), 11)
((0, 0, 10), 11)
((0, 0, 11), 11)
((0, 0, 12), 11)
((0, 0, 13), 11)
((0, 0, 14), 11)
((0, 0, 15), 11)
((0, 0, 16), 11)
((0, 0, 17), 11)
((0, 0, 18), 11)
((0, 0, 19), 11)
((0, 0, 20), 11)
((0, 0, 21), 11)
((0, 0, 22), 11)
((0, 0, 23), 11)
((0, 0, 24), 11)


## Finding the worst numbers in the Miller-Rabin primality test (for `n` in a certain range, 3 witnesses)

In [86]:
# imports
from tqdm import tqdm
import itertools

# constant
DEFENDANT_RANGE = 25
DEFENDANTS = [i for i in range(DEFENDANT_RANGE + 1) if i == 2 or i % 2 != 0]
if DEFENDANT_RANGE % 2 == 0:
  WITNESS_RANGE = DEFENDANT_RANGE - 1
else:
  WITNESS_RANGE = DEFENDANT_RANGE
WITNESSES = list(itertools.product([i for i in range(WITNESS_RANGE)], [i for i in range(WITNESS_RANGE)], [i for i in range(WITNESS_RANGE)], [i for i in range(WITNESS_RANGE)]))

# variables
results = {}

# trial loop function
def trial_loop(defendants, witnesses):
  lie_count = 0

  for defendant in defendants:
      if miller_rabin(defendant, [witnesses[0], witnesses[1]]) != actual_prime(defendant):
        lie_count += 1
  
  return lie_count

# main loop
for i in tqdm(WITNESSES):
  results[i] = trial_loop(DEFENDANTS, (i))

# finding top 25 worst witnesses
for i in sorted(results.items(), key=lambda x: x[1], reverse=True)[:25]:
  print(i)

100%|██████████| 390625/390625 [02:08<00:00, 3049.88it/s]


((0, 0, 0, 0), 5)
((0, 0, 0, 1), 5)
((0, 0, 0, 2), 5)
((0, 0, 0, 3), 5)
((0, 0, 0, 4), 5)
((0, 0, 0, 5), 5)
((0, 0, 0, 6), 5)
((0, 0, 0, 7), 5)
((0, 0, 0, 8), 5)
((0, 0, 0, 9), 5)
((0, 0, 0, 10), 5)
((0, 0, 0, 11), 5)
((0, 0, 0, 12), 5)
((0, 0, 0, 13), 5)
((0, 0, 0, 14), 5)
((0, 0, 0, 15), 5)
((0, 0, 0, 16), 5)
((0, 0, 0, 17), 5)
((0, 0, 0, 18), 5)
((0, 0, 0, 19), 5)
((0, 0, 0, 20), 5)
((0, 0, 0, 21), 5)
((0, 0, 0, 22), 5)
((0, 0, 0, 23), 5)
((0, 0, 0, 24), 5)
